In [60]:
from rethon.ensemble_generation import EnsembleGenerator, SimpleEnsembleGenerator
from rethon.ensemble_generation import create_random_argument_list, standard_model_params_varied_alphas
from rethon.ensemble_generation import random_position_as_set, create_random_arguments2
from rethon.model import StandardPosition, DAGDialecticalStructure, StandardGlobalReflectiveEquilibrium
from os import getcwd

In [63]:
# Sentence pool
n= 7
# Two random dialectical structures with 7 arguments each
n_dialectical_structures = 50
arguments_list = [create_random_arguments2(n_sentences=n, 
                                           n_arguments=7,
                                           n_max_premises=2) for i in range(n_dialectical_structures)]


# Two implementations:
implementations = [{'module_name': 'rethon.model',
                    'position_class_name':'StandardPosition',
                    'dialectical_structure_class_name': 'DAGDialecticalStructure',
                    'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'},
                   {'module_name': 'rethon.model',
                    'position_class_name':'StandardPosition',
                    'dialectical_structure_class_name': 'DAGDialecticalStructure',
                    'reflective_equilibrium_class_name': 'StandardLocalReflectiveEquilibrium'
                   }]

# Two random initial commitments
n_positions = 5
coms_list = [random_position_as_set(n_sentences = n) for i in range(n_positions)]
#print(arguments_list)
#print(coms_list)

model_params = [{
    'neighbourhood_depth': 1,
    'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0],
    'account_penalties': [0.0, 0.3, 1.0, 1.0],
    'weights': {'account': 0.35, 'faithfulness': 0.1, 'systematicity': 0.55}
}]

ensemble_gen = SimpleEnsembleGenerator(arguments_list = arguments_list, 
                                 n_sentence_pool = n,
                                 initial_commitments_list = coms_list,
                                 model_parameters_list = model_params,
                                 implementations = implementations)


# removing items we don't need
ensemble_gen.remove_item('account_penalties')
ensemble_gen.remove_item('faithfulness_penalties')
ensemble_gen.remove_item('weight_account')
ensemble_gen.remove_item('weight_systematicity')
ensemble_gen.remove_item('weight_faithfulness')
ensemble_gen.remove_item('n_random_choices')

# defining the function for the global Optima
def data_global_optima(ensemble_generator):
    re_model = ensemble_generator.reflective_equilibrium()
    if re_model.model_name() == "StandardModel":
        return re_model.global_optima(ensemble_generator.initial_commitments())

ensemble_gen.add_item('global optima', data_global_optima)

#model_runs = list(ensemble_gen.ensemble_iter())
#print(f"Generated {len(model_runs)} ensembles.")
# looping through the ensemble
#for model_run in model_runs:
#    print(f"The model run ended with {model_run.state().last_commitments()} as its final commitments.")
    

ensemble_gen.ensemble_items_to_csv(
                              output_file_name = 'data_output.csv',
                              output_dir_name = getcwd(),
                              archive = False, # save the csv as archived tar.gz
                              save_preliminary_results = True, # will create preliminary csv-data sets 
                              preliminary_results_interval = 50)